In [1]:
import time 
t1 = time.time()

In [2]:
!pip install accelerate

In [3]:
from PIL import Image
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_in_model, dispatch_model

In [4]:
import os

os.mkdir('/kaggle/working/model_weight')

In [5]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

# Load the model
model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True, torch_dtype=torch.float16)

# Specify the directory where you want to save the model
save_directory = "/kaggle/working/model_weight"

# Save the model weights
model.save_pretrained(save_directory)

print(f"Model weights saved to {save_directory}")

# Optionally, you can also save the tokenizer if you need it later
# tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5')
# tokenizer.save_pretrained(save_directory)

# print(f"Tokenizer saved to {save_directory}")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

resampler.py:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

model-00006-of-00007.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Model weights saved to /kaggle/working/model_weight


In [6]:
MODEL_PATH = '/kaggle/working/model_weight' # you can download in advance or use `openbmb/MiniCPM-Llama3-V-2_5`

In [7]:
import torch

def get_gpu_info():
    print("PyTorch GPU Information:")
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        print(f"Number of GPUs available: {gpu_count}")
        for i in range(gpu_count):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("No GPU available. PyTorch is using CPU.")

    print("\nCUDA GPU Information:")
    try:
        import pynvml
        pynvml.nvmlInit()
        gpu_count = pynvml.nvmlDeviceGetCount()
        print(f"Number of GPUs available: {gpu_count}")
        for i in range(gpu_count):
            handle = pynvml.nvmlDeviceGetHandleByIndex(i)
            print(f"GPU {i}: {pynvml.nvmlDeviceGetName(handle).decode('utf-8')}")
        pynvml.nvmlShutdown()
    except ImportError:
        print("pynvml not installed. Install it with 'pip install pynvml' for detailed CUDA info.")
    except pynvml.NVMLError:
        print("Unable to initialize NVIDIA Management Library. Make sure you have NVIDIA drivers installed.")

if __name__ == "__main__":
    get_gpu_info()

PyTorch GPU Information:
Number of GPUs available: 2
GPU 0: Tesla T4
GPU 1: Tesla T4

CUDA GPU Information:
Number of GPUs available: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


In [8]:
max_memory_each_gpu = '12GiB' # Define the maximum memory to use on each gpu, here we suggest using a balanced value, because the weight is not everything, the intermediate activation value also uses GPU memory (10GiB < 16GiB)

gpu_device_ids = [0,1] # Define which gpu to use (now we have two GPUs, each has 16GiB memory)

no_split_module_classes = ["LlamaDecoderLayer"]

max_memory = {
    device_id: max_memory_each_gpu for device_id in gpu_device_ids
}

config = AutoConfig.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True, torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)

with init_empty_weights():
    model = AutoModel.from_config(
        config, 
        torch_dtype=torch.float16, 
        trust_remote_code=True
    )

device_map = infer_auto_device_map(
    model,
    max_memory=max_memory, no_split_module_classes=no_split_module_classes
)

print("auto determined device_map", device_map)

# Here we want to make sure the input and output layer are all on the first gpu to avoid any modifications to original inference script.

device_map["llm.model.embed_tokens"] = 0
device_map["llm.model.layers.0"] = 0
device_map["llm.lm_head"] = 0
device_map["vpm"] = 0
device_map["resampler"] = 0

print("modified device_map", device_map)

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenization_minicpmv_fast.py:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5:
- tokenization_minicpmv_fast.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

auto determined device_map OrderedDict([('llm.model.embed_tokens', 0), ('llm.model.layers.0', 0), ('llm.model.layers.1', 0), ('llm.model.layers.2', 0), ('llm.model.layers.3', 0), ('llm.model.layers.4', 0), ('llm.model.layers.5', 0), ('llm.model.layers.6', 0), ('llm.model.layers.7', 0), ('llm.model.layers.8', 0), ('llm.model.layers.9', 0), ('llm.model.layers.10', 0), ('llm.model.layers.11', 0), ('llm.model.layers.12', 0), ('llm.model.layers.13', 0), ('llm.model.layers.14', 0), ('llm.model.layers.15', 0), ('llm.model.layers.16', 0), ('llm.model.layers.17', 0), ('llm.model.layers.18', 0), ('llm.model.layers.19', 0), ('llm.model.layers.20', 0), ('llm.model.layers.21', 0), ('llm.model.layers.22', 0), ('llm.model.layers.23', 0), ('llm.model.layers.24', 1), ('llm.model.layers.25', 1), ('llm.model.layers.26', 1), ('llm.model.layers.27', 1), ('llm.model.layers.28', 1), ('llm.model.layers.29', 1), ('llm.model.layers.30', 1), ('llm.model.layers.31', 1), ('llm.model.norm', 1), ('llm.model.rotary_e

In [9]:
load_checkpoint_in_model(
    model, 
    MODEL_PATH, 
    device_map=device_map)

model = dispatch_model(
    model, 
    device_map=device_map
)

torch.set_grad_enabled(False)

model.eval()

  0%|          | 0/82 [00:00<?, ?w/s]

  0%|          | 0/104 [00:00<?, ?w/s]

  0%|          | 0/100 [00:00<?, ?w/s]

  0%|          | 0/455 [00:00<?, ?w/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernor

In [10]:
# import os
# import time
# import urllib.request
# from pathlib import Path
# from PIL import Image

# def create_placeholder_image(image_save_path):
#     try:
#         placeholder_image = Image.new('RGB', (100, 100), color='black')
#         placeholder_image.save(image_save_path)
#     except Exception as e:
#         return

# def download_image(image_link, save_folder, retries=3, delay=3):
    
#     if not isinstance(image_link, str):
#         return

#     filename = Path(image_link).name
#     image_save_path = os.path.join(save_folder, filename)

#     print(image_save_path)
#     print("-------------_")
#     if os.path.exists(image_save_path):
#         return

#     for _ in range(retries):
#         try:
#             urllib.request.urlretrieve(image_link, image_save_path)
#             return
#         except Exception as e:
#             print(f"Error downloading {image_link}: {e}")
#             time.sleep(delay)
    
#     create_placeholder_image(image_save_path)

In [11]:
def generate_response(img, instruction_string="Give me the weight of the product with units"):
    with torch.no_grad():
        response = model.chat(
            image=img,
            msgs=[
                {
                    "role": "user",
                    "content": instruction_string
                }
            ],
            tokenizer=tokenizer
        )
    
    return response

In [12]:
# ## if you want to use streaming, please make sure sampling=True and stream=True
# ## the model.chat will return a generator
# res = model.chat(
#     image=image,
#     msgs=msgs,
#     tokenizer=tokenizer,
#     sampling=True,
#     temperature=0.7,
#     stream=True
# )

# generated_text = ""
# for new_text in res:
#     generated_text += new_text
#     print(new_text, flush=True, end='')

<h1>Testing Code</h1>

In [13]:
# download_image("https://m.media-amazon.com/images/I/A1brlRJvdnL.jpg", "/kaggle/working/")

In [14]:
# image_path = '/kaggle/working/A1brlRJvdnL.jpg'

# generate_response(image_path, "Give me the weight of the product with units")

<h1>Main CSV Code</h1>

In [15]:
# import pandas as pd
# import requests
# import os

# def process_csv_and_predict(csv_path, output_csv_path, image_dir):
#     # Read the CSV file
#     df = pd.read_csv(csv_path)
    
#     # Create image directory if it doesn't exist
#     os.makedirs(image_dir, exist_ok=True)
    
#     # Function to download image
#     def download_image(url, filename):
#         response = requests.get(url)
#         with open(filename, 'wb') as f:
#             f.write(response.content)
#         return filename

#     # Process each row
#     predictions = []
#     for index, row in df.iterrows():
#         try:
#             # Download image
#             image_filename = os.path.join(image_dir, f"{row['group_id']}.jpg")
#             downloaded_image_path = download_image(row['image_link'], image_filename)
            
#             # Prepare instruction using entity_name
#             instruction = f"Give me the {row['entity_name']} of the product with units, given to point answer and in short within 15 words"
            
#             # Make prediction using demo_process_vqa
#             prediction_output = generate_response(downloaded_image_path, instruction)
            
#             # Extract the answer from the prediction output
#             answer = prediction_output
            
#             predictions.append(answer)
            
#             os.remove(image_filename)
            
#             print(f"Processed row {index + 1}/{len(df)} with the answer {answer}")

#             # Save output after every 10 predictions
#             if (index + 1) % 10 == 0:
#                 df.loc[:index, 'prediction'] = predictions
#                 df.to_csv(output_csv_path, index=False)
#                 print(f"Saved progress to {output_csv_path} after {index + 1} predictions")

#         except Exception as e:
#             print(f"Error processing row {index + 1}/{len(df)}: {str(e)}")
#             predictions.append("Error")

#     # Add any remaining predictions to dataframe
#     df['prediction'] = predictions
    
#     # Save final updated dataframe
#     df.to_csv(output_csv_path, index=False)
#     print(f"Final updated CSV saved to {output_csv_path}")

# # Usage
# csv_path = '/kaggle/input/amazon-ml-challenge/student_resource 3/dataset/train.csv'
# output_csv_path = '/kaggle/working/output.csv'
# image_dir = 'downloaded_images'
# process_csv_and_predict(csv_path, output_csv_path, image_dir)

In [16]:
# import re
# import os
# import requests
# import pandas as pd
# import multiprocessing
# import time
# from time import time as timer
# from tqdm import tqdm
# import numpy as np
# from pathlib import Path
# from functools import partial
# import requests
# import urllib
# from PIL import Image


# def download_images(image_links, download_folder, allow_multiprocessing=True):
#     if not os.path.exists(download_folder):
#         os.makedirs(download_folder)

#     if allow_multiprocessing:
#         download_image_partial = partial(
#             download_image, save_folder=download_folder, retries=3, delay=3)

#         with multiprocessing.Pool(64) as pool:
#             list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
#             pool.close()
#             pool.join()
#     else:
#         for image_link in tqdm(image_links, total=len(image_links)):
#             download_image(image_link, save_folder=download_folder, retries=3, delay=3)
            
# df = pd.read_csv("/kaggle/input/amazon-ml-challenge/student_resource 3/dataset/train.csv")
# download_images(df['image_link'], "/kaggle/working/images")

In [17]:
import os

os.chdir('/kaggle')

In [18]:
## import pandas as pd
import requests
import os
import multiprocessing
from functools import partial
import torch
from io import BytesIO
import time
import pandas as pd


# Set the start method to 'spawn'
multiprocessing.set_start_method('spawn', force=True)

def download_image(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

def process_row(row, image_dir):
    try:
        image_filename = os.path.join(image_dir, f"{row['group_id']}.jpg")
        downloaded_image = download_image(row['image_link'])
        
        instruction = f"Give me the {row['entity_name']} of the product with units, given to point answer and in short within 5 words only"
        
        # Assuming generate_response is a function that uses CUDA
        prediction_output = generate_response(downloaded_image, instruction)
        
#         os.remove(image_filename)
        
#         print(f"Processed row {row.name + 1} with the answer {prediction_output}")
        return prediction_output
    except Exception as e:
        print(f"Error processing row {row.name + 1}: {str(e)}")
        return "Error"

def process_batch(batch, image_dir):
    results = []
    for _, row in batch.iterrows():
        results.append(process_row(row, image_dir))
    return results

def process_csv_and_predict(csv_path, output_csv_path, image_dir,start,stop, batch_size=20):
    df = pd.read_csv(csv_path)
    df = df[start:stop].reset_index(drop=True)
    os.makedirs(image_dir, exist_ok=True)
    
    predictions = []
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch_predictions = process_batch(batch, image_dir)
        predictions.extend(batch_predictions)
#         print(len(df.loc[i:i+batch_size-1, 'prediction']))
#         print(i,i+batch_size-1)
        print(len(batch_predictions))
        df.loc[i:i+batch_size-1, 'prediction'] = batch_predictions
        df.to_csv(output_csv_path, index=False)
        print(f"Saved progress to {output_csv_path} after {i + len(batch_predictions)} predictions")
        print()
        if (time.time() - t1) > 42000:
            break
    
    print(f"Final updated CSV saved to {output_csv_path}")

if __name__ == '__main__':
    t2 = time.time()
    start_idx = 11001
    stop_idx = 18001
    csv_path = '/kaggle/input/amazon-ml-challenge/student_resource 3/dataset/test.csv'
    output_csv_path = '/kaggle/working/output.csv'
    image_dir = 'downloaded_images'
    process_csv_and_predict(csv_path, output_csv_path, image_dir,start_idx,stop_idx)
    print(f"Time taken is {time.time()-t2} seconds")

preprocessor_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

processing_minicpmv.py:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

image_processing_minicpmv.py:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5:
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5:
- processing_minicpmv.py
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


20
Saved progress to /kaggle/working/output.csv after 20 predictions



...

20
Saved progress to /kaggle/working/output.csv after 6960 predictions



/opt/conda/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


20
Saved progress to /kaggle/working/output.csv after 6980 predictions



/opt/conda/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


20
Saved progress to /kaggle/working/output.csv after 7000 predictions

Final updated CSV saved to /kaggle/working/output.csv
Time taken is 31343.19691681862 seconds
